In [1]:
from copy import deepcopy
from dataclasses import dataclass
import multiprocessing as mp
from re import T
import torchvision.models as models
import torch
import copy
import torch.nn.functional as F
#from model import net
import random
import multiprocessing as mp
import os
import os
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
from torchvision import datasets,transforms
import random
from torch import optim,ceil
import torch.nn.functional as F
from torch import nn
import pickle
import numpy as np
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from pandas import DataFrame
from torchmetrics.classification import MulticlassAccuracy


In [2]:

from torch.utils.data import Dataset,DataLoader
import numpy as np

class plantVillage(Dataset):
    def __init__(self, df,
                  img_dir='data/oneFolder'
                 , isTest=False):
        self.img_labels=df
        self.img_dir='../../Data/PlantVillage/PlantVillage-Dataset-master/All/resized512/'
       # train, test = train_test_split(img_labels, test_size=0.05)

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
            img_path = os.path.join(self.img_dir, self.img_labels['image'][idx])

            image = (read_image(img_path).float())
            transform=transforms.Compose([transforms.Resize((256,256)),
        transforms.Normalize((-0.7898, -1.4373, -0.6051), (1.3209, 1.5867, 1.1863))])
            image = transform(image)
            label = self.img_labels['disease'][idx]
            return image, label

In [3]:
df=pd.read_csv('./plantVillage.csv')
df=df.drop('Unnamed: 0',axis=1)
train,valid_test=train_test_split(df,test_size=0.2)
valid,test=train_test_split(valid_test,test_size=0.5)

train_data=plantVillage(df=train.reset_index())
test_data=plantVillage(df=test.reset_index())
valid_data=plantVillage(df=valid.reset_index())

model=models.mobilenet_v2(pretrained=True)
model.classifier[1]=(torch.nn.Linear(in_features=1280, out_features=21, bias=True))

In [4]:


def client_trainGPU(config):
    model=models.mobilenet_v2(pretrained=True)
    model.classifier[1]=(torch.nn.Linear(in_features=1280, out_features=21, bias=True))

    dataset=test_data
    test_dataset=test_data
    criterion=torch.nn.CrossEntropyLoss()
    id=1
    optimizer=optim.Adam(model.parameters(),lr=config['learning_rate'],weight_decay=config['weight_decay'])
    dataloader=torch.utils.data.DataLoader(dataset,
                                        batch_size=int(config['batch_size']),
                                        shuffle=True)

    train_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    model.to(train_device)

    print('Client {} start Training'.format(id))

    
    for epoc in (range(config['epoch'])):
        epoch_loss=0.0
        step=0
        epoc_accuracy=[]

        count=0
        for x,y in iter(dataloader):
            x,y=x.to(train_device),y.to(train_device)
            output=model(x)

            loss=criterion(output,y)

            x,y=x.cpu(),y.cpu()
            output=output.cpu().detach()
            epoch_loss+=loss.item()
            step+=1
            pred=output.argmax(dim=1,keepdim=True).T[0]
            
            epoc_accuracy.append(MulticlassAccuracy(num_classes=21)(pred,y).item())
            
            optimizer.zero_grad(set_to_none=True)
            loss.backward()
            optimizer.step()
            count+=1

        accuracy=np.mean(epoc_accuracy)
        print('Epoch : {} , loss: {}, accuracy: {}'.format(epoc,(epoch_loss/step),accuracy))

        wandb.log({
                        'epoc':epoc,
                        'loss':(epoch_loss/step),
                        'accuracy':accuracy
                        })
    model=model.cpu()
    print("Finished Training")

       



In [5]:
#Test
# config={
#         'batch_size': 16,
#         'epoch':5,
#         'learning_rate': 0.01,
#         'weight_decay':0.00001
# }
# client_trainGPU(config)


In [6]:
import wandb
import numpy as np 
import random

# Define sweep config
sweep_configuration = {
    'method': 'random',
    'name': 'sweep',
    'metric': {'goal': 'maximize', 'name': 'val_acc'},
    'parameters': 
    {
        'batch_size': {'values': [16]},
        'epoch': {'values': [5]},
        'learning_rate': {'max': 0.01, 'min': 0.0001},
        'weight_decay':{'max': 0.00005, 'min': 0.00001}
     }
}
sweep_id = wandb.sweep(sweep=sweep_configuration, project='modeltunning_village_5')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: wgjbj2xu
Sweep URL: https://wandb.ai/makkuror/modeltunning_village_5/sweeps/wgjbj2xu


In [7]:
os.environ["WANDB_API_KEY"] = '761f77141558010f4d706f726fcf122aac0aae23'
os.environ["WANDB_MODE"]="offline"
def main():

    run = wandb.init()
    
    # note that we define values from `wandb.config` instead 
    # of defining hard values

    config = {
        "learning_rate": wandb.config.learning_rate,
        "batch_size":16,
        "epoch":wandb.config.epoch,
        "weight_decay":wandb.config.weight_decay
    }
    import warnings 
    warnings.filterwarnings('ignore')
    a=client_trainGPU(config)



# Start sweep job.
wandb.agent(sweep_id, function=main, count=50)

wandb: Agent Starting Run: f40rq71r with config:
wandb: 	batch_size: 16
wandb: 	epoch: 5
wandb: 	learning_rate: 0.0016111222038979367
wandb: 	weight_decay: 0.0004791203813681829
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Client 1 start Training
Epoch : 0 , loss: 1.075828147983516, accuracy: 0.24690705508169264
Epoch : 1 , loss: 0.6491373887666898, accuracy: 0.308093289552539
Epoch : 2 , loss: 0.5515740046028009, accuracy: 0.32554916438368176
Epoch : 3 , loss: 0.47301045750900117, accuracy: 0.3431398166077478
Epoch : 4 , loss: 0.42559778963562667, accuracy: 0.349639109257658
Finished Training


accuracy,▁▅▆██
epoc,▁▃▅▆█
loss,█▃▂▂▁
accuracy,0.34964
epoc,4
loss,0.4256


wandb: Agent Starting Run: 7ykpqcg1 with config:
wandb: 	batch_size: 16
wandb: 	epoch: 5
wandb: 	learning_rate: 0.00700610882600019
wandb: 	weight_decay: 0.00015323164869598878
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Client 1 start Training
Epoch : 0 , loss: 2.4064900086800374, accuracy: 0.06778623425951763
Epoch : 1 , loss: 1.9638191440495194, accuracy: 0.10276750384642028
Epoch : 2 , loss: 1.6559015410638172, accuracy: 0.14371234681374984
Epoch : 3 , loss: 1.431527555339993, accuracy: 0.1771629805757564
Epoch : 4 , loss: 1.2664785771815872, accuracy: 0.20608916868276028
Finished Training


...

Client 1 start Training
Epoch : 0 , loss: 1.6889426336899422, accuracy: 0.1567174720966061
Epoch : 1 , loss: 1.1971473216308235, accuracy: 0.21149088110944836
Epoch : 2 , loss: 0.9892658271740392, accuracy: 0.2456166519068829
Epoch : 3 , loss: 0.895564217937888, accuracy: 0.25926725216352536
Epoch : 4 , loss: 0.8143612644765443, accuracy: 0.27467013891122477
Finished Training


accuracy,▁▄▆▇█
epoc,▁▃▅▆█
loss,█▄▂▂▁
accuracy,0.27467
epoc,4
loss,0.81436


wandb: Agent Starting Run: 4zosc905 with config:
wandb: 	batch_size: 16
wandb: 	epoch: 5
wandb: 	learning_rate: 0.0085082046604511
wandb: 	weight_decay: 0.00010123565233949028
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Client 1 start Training
Epoch : 0 , loss: 2.283386776303332, accuracy: 0.08892974566904188
Epoch : 1 , loss: 1.8335439565431035, accuracy: 0.12787446232076422
Epoch : 2 , loss: 1.591239294001561, accuracy: 0.15776407810830578
Epoch : 3 , loss: 1.3440545128441699, accuracy: 0.1925456022873894
Epoch : 4 , loss: 1.2248425870826565, accuracy: 0.21179561523257018
Finished Training


accuracy,▁▃▅▇█
epoc,▁▃▅▆█
loss,█▅▃▂▁
accuracy,0.2118
epoc,4
loss,1.22484


# Clear Memory GPU

In [8]:
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()


free_gpu_cache()

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  2% | 57% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  6% | 11% |


In [9]:
cf_matrix=0

def confusion_matrix(target,pred):
    for i in range(0,target.shape[0]):
        cf_matrix+=confusion_matrix(target[i], pred[i])
def f1_score(target,pred):
    for i in range(0,target.shape[0]):
       print(accuracy_score(target[i], pred[i]))
f1_score(a,b)

NameError: name 'a' is not defined